# Breaking Down UVA Course Reviews

By: William Kaiser

## Layout

The layout for this site was designed in [Figma](https://www.figma.com/file/PCZFfmRXn0e6720BjxNAtg/Misc-Images?type=design&node-id=539-5&mode=design&t=4der6LRc6aprI2Sf-0) and designed to be responsive and intent-based.

![Layout](./imgs/layout.png)

However, after discussion with fellow class members and looking at [Python Graph Gallery](https://python-graph-gallery.com/) I decided to use a [Spider / Radar](https://python-graph-gallery.com/radar-chart/) plot as well as the more traditional bar charts.

# Data Loading

The data for this analysis comes from [theCourseForum](https://thecourseforum.com/) and a direct database connection was used. 

For more information about data provenance, collection, formatting, and cleaning, please see [Sprint 2: Data](./sprints/sprint2-data.ipynb).

In [33]:
# Generic imports
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from tqdm import tqdm
from openai import OpenAI
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from dash import dash_table, dcc, Dash, html, callback, Input, Output, State
from typing import List, Dict, TypedDict
import os

tqdm.pandas()

In [2]:
# Loading the data
df = pd.read_csv("./data.csv")
df['course_level'] = pd.to_numeric(df['course_number']).apply(lambda x: x // 1000)
df

/var/folders/m0/sl2yvpv551gcfbk3gdntf74r0000gn/T/ipykernel_78167/2655894733.py:2: DtypeWarning: Columns (14,15,26,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data.csv")


,course_id,title,description,course_number,average,a_plus,a,a_minus,b_plus,b,...,amount_group,amount_homework,review_created,year,season,instructor_email,instructor_name,year_last_taught,season_last_taught,course_level
0,12.0,Introductory Accounting I,Designed to introduce students to the language...,2010.0,3.200437,59.0,131.0,51.0,42.0,62.0,...,4,2.0,2020-11-10 22:11:29+00:00,2020,FALL,gdb5x@virginia.edu,Gary Brooks,2023.0,SPRING,2.0
1,1505.0,History and Civilization of Classical India,Studies the major elements of South Asian civi...,2001.0,3.395795,5.0,85.0,132.0,117.0,66.0,...,0,0.0,2020-12-19 04:08:23.637939+00:00,2020,FALL,sal9c@virginia.edu,Spencer Leonard,2021.0,FALL,2.0
2,1504.0,Introductory Seminar in South Asia,Introduction to the study of history intended ...,1501.0,3.601403,9.0,118.0,131.0,74.0,27.0,...,0,0.0,2016-11-02 16:38:11+00:00,2016,FALL,sal9c@virginia.edu,Spencer Leonard,2022.0,SPRING,1.0
3,1505.0,History and Civilization of Classical India,Studies the major elements of South Asian civi...,2001.0,3.395795,5.0,85.0,132.0,117.0,66.0,...,0,0.0,2021-12-20 18:54:54.439844+00:00,2021,FALL,sal9c@virginia.edu,Spencer Leonard,2021.0,FALL,2.0
4,15083.0,India From Akbar to Victoria,Studies the society and politics in the Mughal...,3002.0,2.984615,0.0,2.0,6.0,1.0,1.0,...,0,0.0,2022-11-03 03:15:03.409391+00:00,2021,FALL,sal9c@virginia.edu,Spencer Leonard,2021.0,FALL,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41507,754.0,Money and Banking,Studies the role of money in the economic syst...,3030.0,3.144924,136.0,754.0,742.0,435.0,596.0,...,0,0.0,2011-06-27 00:00:00+00:00,2009,SPRING,hjm5p@virginia.edu,Hernan Moscoso Boedo,2024.0,SPRING,3.0
41508,754.0,Money and Banking,Studies the role of money in the economic syst...,3030.0,3.144924,136.0,754.0,742.0,435.0,596.0,...,0,0.0,2013-03-03 00:00:00+00:00,2009,SPRING,hjm5p@virginia.edu,Hernan Moscoso Boedo,2024.0,SPRING,3.0
41509,754.0,Money and Banking,Studies the role of money in the economic syst...,3030.0,3.144924,136.0,754.0,742.0,435.0,596.0,...,0,1.0,2013-08-15 00:00:00+00:00,2009,SPRING,hjm5p@virginia.edu,Hernan Moscoso Boedo,2024.0,SPRING,3.0
41510,8600.0,Special Topics in Politics,Special Topics in Politics,2500.0,3.419390,11.0,81.0,71.0,42.0,44.0,...,0,0.0,2021-01-20 19:38:10.579957+00:00,2021,JANUARY,dal7w@virginia.edu,David Leblang,2024.0,SPRING,2.0


In [3]:
# Printing the columns
print("\n".join(df.columns.to_list()))

course_id
title
description
course_number
average
a_plus
a
a_minus
b_plus
b
b_minus
c_plus
c
c_minus
total_enrolled
dfw
subdepartment_name
mnemonic
review_text
instructor_rating
difficulty
recommendability
enjoyability
hours_per_week
amount_reading
amount_writing
amount_group
amount_homework
review_created
year
season
instructor_email
instructor_name
year_last_taught
season_last_taught
course_level


## Making the Spider Plot

The goal of this section is for a specified course to be selected and then a radar plot to be generated based on the reviews for that course.

In [4]:
REVIEW_COMPONENTS = {
    "instructor_rating": "Instructor Rating",
    "difficulty": "Difficulty",
    "recommendability": "Recommendability",
    "enjoyability": "Enjoyability",
    # "hours_per_week": "Hours Per Week",
    "amount_reading": "Amount Reading",
    "amount_writing": "Amount Writing",
    "amount_group": "Amount Groupwork",
    "amount_homework": "Amount Homework",
}

# making each column into a numeric column
for column in REVIEW_COMPONENTS.keys():
    df[column] = pd.to_numeric(df[column], errors="coerce")

COURSE = "CS 3100"

def course_components(course_pneumonic_and_number: str) -> (str, int):
    """
    Turns a human readable course pneumonic and number into a database-searchable tuple
    """
    course_pneumonic_and_number = (
        course_pneumonic_and_number.strip().upper().replace("  ", " ").split(" ")
    )
    pneumoic, number = course_pneumonic_and_number
    print(pneumoic, number)
    return pneumoic, int(number)


def get_course_summary_ratings(pneumonic: str, number: int) -> pd.DataFrame:
    """
    Gets the ratings for the course
    """
    pneumonic_mask = df["mnemonic"] == pneumonic
    number_mask = df["course_number"] == number
    relevant = df[pneumonic_mask & number_mask]

    mean = relevant[REVIEW_COMPONENTS.keys()].mean()
    std = relevant[REVIEW_COMPONENTS.keys()].std()

    frame = pd.DataFrame(
        {"mean": mean.to_list(), "std": std.to_list(), "category": mean.index.to_list()}
    )
    frame['course'] = frame['mean'].apply(lambda x: f"{pneumonic} {number}")
    return frame


get_course_summary_ratings(*course_components(COURSE))

CS 3100


,mean,std,category,course
0,4.058824,1.344925,instructor_rating,CS 3100
1,4.235294,0.752447,difficulty,CS 3100
2,3.588235,1.416811,recommendability,CS 3100
3,3.411765,1.416811,enjoyability,CS 3100
4,1.235294,2.411675,amount_reading,CS 3100
5,2.058824,2.988950,amount_writing,CS 3100
6,1.705882,2.687115,amount_group,CS 3100
7,3.470588,4.079180,amount_homework,CS 3100


In [5]:
# Creating a spider plot for the course
# source: https://python-graph-gallery.com/571-radar-chart-with-plotly/
# Getting many plot with px
# source: https://stackoverflow.com/questions/56727843/how-can-i-create-subplots-with-plotly-express
# note: this sucks


def course_axis(
    frame: pd.DataFrame, course: str = None, include: str = "ratings"
) -> px.line_polar:
    """
    Makes a polar axis to compare courses
    """
    if "duration" in include:
        frame = frame[frame["category"].apply(lambda name: name.find("amount") != -1)]
    if "rating" in include:
        frame = frame[frame["category"].apply(lambda name: name.find("amount") == -1)]

    fig = go.Figure()
    frame['category'] = frame['category'].replace(REVIEW_COMPONENTS)
    for course in frame["course"].unique():
        rel_frame = frame[frame["course"] == course]
        graph_object = go.Scatterpolar(
            r=rel_frame["mean"], theta=rel_frame["category"], fill="toself",
             name=course.upper()
        )

        fig.add_trace(graph_object)

    return fig


cs_3100 = get_course_summary_ratings(*course_components(COURSE))
cs_2130 = get_course_summary_ratings(*course_components("CS 2130"))
cs_3130 = get_course_summary_ratings(*course_components("CS 3130"))
ds_4003 = get_course_summary_ratings(*course_components("DS 4003"))


rows = pd.concat([cs_3100, cs_2130, cs_3130, ds_4003])

course_axis(rows, course="CS 3100")

CS 3100
CS 2130
CS 3130
DS 4003


/var/folders/m0/sl2yvpv551gcfbk3gdntf74r0000gn/T/ipykernel_78167/3246814276.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Excel-Style Filterable Table

The goal of this section is to allow user filtering of tables in an excel like manner. 

The data in the section will be similar to [the spider plot](#making-the-spider-plot), however the format will be a bit more formal.

![Excel-style Table](./imgs/excel-table.png)

**Guide:** https://dash.plotly.com/datatable

In [6]:
# Defining columns of interest
SAMPLE_COURSES = ["CS 3100", "CS 2130", "CS 3130", "DS 4003"]

# additional table components
TABLE_COMPONENTS = {
    'average': 'Average',
}

course_components_to_agg = {**TABLE_COMPONENTS, **REVIEW_COMPONENTS}
course_components_to_agg

{'average': 'Average',
 'instructor_rating': 'Instructor Rating',
 'difficulty': 'Difficulty',
 'recommendability': 'Recommendability',
 'enjoyability': 'Enjoyability',
 'amount_reading': 'Amount Reading',
 'amount_writing': 'Amount Writing',
 'amount_group': 'Amount Groupwork',
 'amount_homework': 'Amount Homework'}

In [7]:
def get_data_for_course_comparison_table(courses: List) -> pd.DataFrame:
    """
    Gets relevant course_components to aggregate for a list of courses
    """
    print(courses)

    # filtering the data down to the relevant frame
    mask = df["mnemonic"] == -1
    for course in courses:
        pneumonic, number = course
        mask = mask | ((df["mnemonic"] == pneumonic) & (df["course_number"] == number))
    
    relevant = df[mask]

    grouped = relevant.groupby(["mnemonic", "course_number"]).agg({ key: 'mean' for key in course_components_to_agg.keys() }).reset_index()

    return grouped

course_tuples = [course_components(course) for course in SAMPLE_COURSES]
course_data = get_data_for_course_comparison_table(course_tuples)
course_data

CS 3100
CS 2130
CS 3130
DS 4003
[('CS', 3100), ('CS', 2130), ('CS', 3130), ('DS', 4003)]


,mnemonic,course_number,average,instructor_rating,difficulty,recommendability,enjoyability,amount_reading,amount_writing,amount_group,amount_homework
0,CS,2130.0,2.948611,3.450000,4.300000,2.500000,2.300000,1.900000,1.200000,1.150000,5.700000
1,CS,3100.0,3.379770,4.058824,4.235294,3.588235,3.411765,1.235294,2.058824,1.705882,3.470588
2,CS,3130.0,3.330000,3.600000,5.000000,2.600000,2.300000,1.500000,7.500000,0.300000,2.500000


In [8]:
# making the table in plotly
# source: https://plotly.com/python/table/

course_components_to_agg = {
    **course_components_to_agg,
    "course_number": "Course #",
    "mnemonic": "Mnemonic",
}


def get_cells(course_data: pd.DataFrame) -> List[List[any]]:
    """
    Gets the cells in the data table
    """
    return course_data.round(2).to_numpy().transpose().tolist()


pretty_column_names = list(
    map(
        lambda col: f"<b>{course_components_to_agg.get(col, col).replace(' ', '<br>')}</b><br>",
        course_data.columns.to_list(),
    )
)


table = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=pretty_column_names,
                line_color="darkslategray",
                fill_color="royalblue",
                align=["left", "center"],
                font=dict(color="white", size=12),
                height=40,
            ),
            cells=dict(
                values=get_cells(course_data),
                line_color="darkslategray",
                fill=dict(color=["paleturquoise", "white"]),
                align=["left", "center"],
                font_size=12,
                height=30,
            ),
        )
    ]
)

table  # note: I am pretty happy with this. I would like to add more interactivity here.

In [9]:
# making a source dropdown
# source: https://dash.plotly.com/dash-core-components/dropdown
# df['course_number'] = df['course_number'].apply(int)
df['course_number'].fillna(0, inplace=True)
df['name'] = df['mnemonic'] + " " + df['course_number'].astype(str)
df['name']

0        ACCT 2010.0
1        HISA 2001.0
2        HISA 1501.0
3        HISA 2001.0
4        HISA 3002.0
            ...     
41507    ECON 3030.0
41508    ECON 3030.0
41509    ECON 3030.0
41510    PLAD 2500.0
41511    COMM 3845.0
Name: name, Length: 41512, dtype: object

In [10]:
# adding source dropdowns here
course_mneumoic_dropdown = dcc.Dropdown(
    id='course_dropdown',
    options=[{'label': str(course).replace(".0", ""), 'value': str(course).replace(".0", "")} for course in df['name'].unique()],
    value='CS 3100',
    multi=True
)


# Breaking Down Instructor Rating

![Correlating Instructor Reviews](./imgs/correlating_reviews.png)

In [11]:
# the goal for this is to create the functions which can control for a  particular factor
CHECKBOX_SELECTORS = {
    'average': 'GPA',
    # 'course_level': 'Course Level (1k, 2k, ...)',
    'difficulty' : 'Difficulty',
    'hours_per_week': 'Hours Worked Per Week',
    'amount_group': 'Group Work Per Week'
}

In [12]:
# making a bunch of checkboxes in plotly
checklist = dcc.Checklist(
    options=[
        {'label': v, 'value': k} for k, v in CHECKBOX_SELECTORS.items()
    ],
    value=[],
    id='features_to_plot'
)

In [13]:
# creating a histogram of course reviews
course_reviews = px.histogram(
    df.groupby(["mnemonic", "course_number"]).agg({"instructor_rating": "mean"}),
    x="instructor_rating",
)
course_reviews

### Making a regression

In [14]:
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# making a linear regression model
model = LinearRegression()

columns = ['difficulty', 'instructor_rating', 'hours_per_week']
# getting the data
new_columns = ['average', 'difficulty', 'hours_per_week', 'amount_group']
df.dropna(subset=new_columns, inplace=True)
X = df[new_columns].values.reshape(-1, len(new_columns))
Y = df['instructor_rating'].values.reshape(-1, 1)

# fitting the model
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

model.fit(X_train, Y_train)

print(model.coef_, model.intercept_)

[[ 0.34206444 -0.19451242 -0.00778752 -0.03334443]] [3.26735069]


In [15]:
@callback(
    Output('correlation_plot', 'figure'),
    Input('features_to_plot', 'value')
)
def make_correlation_plot(columns): 
    """
    Makes a bar chart of the correlation between columns
    """
    # uses the data from the model
    data = zip(new_columns, model.coef_[0])
    print(data)
    # filter the data by which the column is in the columns
    column_renamer = {
        'average': 'GPA',
        'difficulty': 'Difficulty',
        'hours_per_week': 'Hours Worked Per Week',
        'amount_group': 'Group Work Per Week'
    }

    x_labels = []
    y_values = []
    for col, r in data:
        if col in columns:
            x_labels.append(column_renamer.get(col, col))
            y_values.append(r)
    
    fig = px.bar(x=x_labels, y=y_values, labels= {'x': 'Feature', 'y': 'Correlation Coefficient', **column_renamer})
    return fig

make_correlation_plot(new_columns)

In [16]:
@callback(
    Output('review_residuals', 'figure'),
    Input('features_to_plot', 'value')
)
def make_residual_plot(columns): 
    """
    Makes a bar chart of the correlation between columns
    """
    data = zip(new_columns, model.coef_[0])
    print(data)
    # filter the data by which the column is in the columns
    column_renamer = {
        'average': 'GPA',
        'difficulty': 'Difficulty',
        'hours_per_week': 'Hours Worked Per Week',
        'amount_group': 'Group Work Per Week'
    }
    print("Columns to plot", columns)

    x_labels = []
    y_values = []
    for col, r in data:
        if col in columns:
            x_labels.append(column_renamer.get(col, col))
            y_values.append(r)

    new_df = df
    for col in new_columns:
        if col not in columns:
            new_df[col] = df[col].apply(lambda x: 0)
            print("Zeroing out", col)

    residuals = pd.Series(model.predict(new_df[new_columns]).flatten()) - new_df['instructor_rating']
    residuals = residuals.dropna()
    fig = px.histogram(x=residuals, title="Review Residuals", labels={
        "count": "# of Reviews",
        "x": "Difference between predicted and actual review"
    })

    fig.update_layout(

    )

    # making a plot of reviews
    return fig


make_residual_plot(new_columns)

Columns to plot ['average', 'difficulty', 'hours_per_week', 'amount_group']


/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



In [17]:
model.intercept_, model.get_params(), model.coef_

(array([3.26735069]),
 {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False},
 array([[ 0.34206444, -0.19451242, -0.00778752, -0.03334443]]))

In [18]:
# getting measures of importance

from sklearn.inspection import permutation_importance

result = permutation_importance(model, X_test, Y_test, n_repeats=10, random_state=42)

X_train

array([[3.42568935, 2.        , 7.        , 0.        ],
       [3.40264617, 3.        , 5.        , 1.        ],
       [3.49704288, 2.        , 2.        , 0.        ],
       ...,
       [3.36904232, 4.        , 6.        , 0.        ],
       [3.5097039 , 2.        , 1.        , 0.        ],
       [2.99779164, 5.        , 0.        , 0.        ]])

## Building Semantic Search

Semantic search will take place with an OpenAI client and a embedding model. This will use a pinecone database for textual similarity.

![Semantic Search](./imgs/semantic-search.png)

In [31]:
# getting and saving course reviews
course_reviews = df.groupby(['title', 'description']).agg({'instructor_rating': 'mean'}).reset_index()

def make_prompt(row):
    """
    Makes a prompt for the user to ask the course
    """

    return f"""course name {row['title']} ({row['description']}) has an average rating of {row['instructor_rating']}"""

course_reviews['prompt'] = course_reviews.apply(make_prompt, axis=1)
course_reviews['prompt']

0       course name "Spiritual But Not Religious": Spi...
1       course name 1492 and the Aftermath (Examines S...
2       course name 17th Century Philosophy (Studies t...
3       course name 18th Century Philosophy (Studies t...
4       course name A Buddhist Approach to Development...
                              ...                        
2386    course name Writing and Critical Inquiry: Comm...
2387    course name Writing with Sound (This course tr...
2388    course name Writing with Style (Develops an un...
2389    course name Young Adult Literature (Using Sims...
2390    course name Zen (Studies the development and h...
Name: prompt, Length: 2391, dtype: object

In [34]:
# getting the embeddings from OpenAI
# from: https://platform.openai.com/docs/guides/embeddings/use-cases

client = OpenAI()

# def get_embedding(text, model="text-embedding-3-small"):
#    text = text.replace("\n", " ")
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

# course_reviews['ada_embedding'] = course_reviews['prompt'].progress_apply(lambda x: get_embedding(x, model='text-embedding-3-small'))
# course_reviews.to_csv('data_1k_embeddings.csv', index=False)

100%|██████████| 2391/2391 [07:39<00:00,  5.20it/s]


In [36]:
# from pinecone import Pinecone, ServerlessSpec
# import os

# # initialize connection to pinecone (get API key at app.pc.io)
# api_key = os.environ.get('PINECONE_API_KEY')

# # configure client
# pc = Pinecone(api_key=api_key)
# spec = ServerlessSpec(cloud='aws', region='us-west-2')

# try:
#     pc.create_index(
#         name="course-reviews-1k", 
#         dimension=1536, 
#         metric="euclidean",
#         spec=spec
#     )
# except:
#     pass # index already created

In [45]:
# index = pc.Index("course-reviews-1k")

# def force_ascii(string: str) -> str:
#     """
#     Forces a string to be ascii
#     """
#     return string.encode('ascii', errors='ignore').decode('ascii')

# # making the things to upsert
# upsertion_reviews = [
#     {"id": force_ascii(item["title"]), "values": item["ada_embedding"]}
#     for item in course_reviews.to_dict(orient="records")
# ]

# # upload loop
# for i in range(0, len(upsertion_reviews), 100):
#     print(f"Uploading {i} to {i + 100}")
#     index.upsert(vectors=upsertion_reviews[i:i + 100])

Uploading 0 to 100
Uploading 100 to 200
Uploading 200 to 300
Uploading 300 to 400
Uploading 400 to 500
Uploading 500 to 600
Uploading 600 to 700
Uploading 700 to 800
Uploading 800 to 900
Uploading 900 to 1000
Uploading 1000 to 1100
Uploading 1100 to 1200
Uploading 1200 to 1300
Uploading 1300 to 1400
Uploading 1400 to 1500
Uploading 1500 to 1600
Uploading 1600 to 1700
Uploading 1700 to 1800
Uploading 1800 to 1900
Uploading 1900 to 2000
Uploading 2000 to 2100
Uploading 2100 to 2200
Uploading 2200 to 2300
Uploading 2300 to 2400


## Final Dashboard

Where all of the components come together to make the final dashboard. This launches the layout and the final project.

In [19]:
# defining callbacks
@callback(
    Output(component_id="course_axis", component_property="figure"),
    (Input(component_id="course_dropdown", component_property="value"))
)
def update_course_axis(course: str):
    """
    Updates the course axis
    """
    if course is str: # if the course is a string
        course = [course]
    
    summary = pd.concat([get_course_summary_ratings(*course_components(course)) for course in course])
    
    # rows = pd.concat([cs_3100, cs_2130, cs_3130, ds_4003])

    # course_data = get_course_summary_ratings(*course_components(course))
    return course_axis(summary, course=course)

@callback(
    Output(component_id="course_hours", component_property="figure"),
    (Input(component_id="course_dropdown", component_property="value"))
)
def update_course_hours(course: str):
    """
    Updates the course hours
    """
    if course is str:
        course = [course]
    
    summary = pd.concat([get_course_summary_ratings(*course_components(course)) for course in course])
    return course_axis(summary, course=course, include="duration")


### Training a linear model to predict the instructor rating based on (GPA, Course Level, Difficulty, Hours Worked Per Week, Group Work Per Week)


In [ ]:
# Making an app to display everything
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

### HEADER ###
header = html.Div([
    html.H1("Course Review Explorer"),
])

### BREAKING DOWN RATINGS ###
course_filter_header = html.Div([
    html.H1("Breaking down ratings"),
    html.P("Examine course reviews for a particular class"),
])

course_filter_row = html.Div([
     dcc.Graph(figure=course_axis(rows, course="CS 3100"), id="course_axis", className="one-half column"),
     dcc.Graph(figure=course_axis(rows, course="CS 2130"), id="course_hours", className="one-half column"),
     dcc.Graph(figure=table, id="course_table", className="one-half column"),
], className="row")

course_filter_section = html.Div([
    course_filter_header,
    course_mneumoic_dropdown,
    course_filter_row,
])

### CORRELATING INSTRUCTOR REVIEWS ###
controlling_factors = html.Div([
    html.H1("Correlating Instructor Reviews"),
    html.Div("Control for the following factors"),
    checklist,
], className="one-third column")

distribution_of_reviews = html.Div([
    html.H1("Distribution of Residuals"),
    html.P("What's left over?"),
    dcc.Graph(id="review_residuals"),
], className="one-third column")

predictive_power_of_features = html.Div([
    html.H1("Predictive Power of Features"),
    html.P("How well do these features predict the instructor rating?"),
    dcc.Graph(id="correlation_plot"),
], className="one-third column")

correlating_instructor_reviews = html.Div([
    controlling_factors,
    distribution_of_reviews,
    predictive_power_of_features
], className="row")

### FINAL LAYOUT ###
app.layout = html.Div([
   header,
   course_filter_section,
   correlating_instructor_reviews,
], className="container")

server = app.server

if __name__ == '__main__':
    app.run_server(jupyter_mode='tab')


Dash app running on http://127.0.0.1:8050/


/var/folders/m0/sl2yvpv551gcfbk3gdntf74r0000gn/T/ipykernel_78167/3246814276.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/m0/sl2yvpv551gcfbk3gdntf74r0000gn/T/ipykernel_78167/3246814276.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<IPython.core.display.Javascript object>

/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but LinearRegression was fitted without feature names

[2024-04-22 17:36:44,249] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/flask/app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/flask/app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/flask/app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/flask/app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/Users/wkaise

Columns to plot []
Zeroing out average
Zeroing out difficulty
Zeroing out hours_per_week
Zeroing out amount_group
Columns to plot ['average']
Zeroing out difficulty
Zeroing out hours_per_week
Zeroing out amount_group


/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



Columns to plot ['average', 'difficulty']
Zeroing out hours_per_week
Zeroing out amount_group


/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but LinearRegression was fitted without feature names



Columns to plot ['average', 'difficulty', 'hours_per_week']
Zeroing out amount_group


/Users/wkaiser/miniforge3/envs/tf/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning:

X has feature names, but LinearRegression was fitted without feature names

